# MISC


In [5]:
import openai
import os

messages = [
    {
        "role": "user",
        "content": "hi",
    },
]

model = "gpt-35-turbo"
client = openai.AzureOpenAI(
    api_key=os.environ.get("AZURE_API_KEY"),
    api_version=os.environ.get("API_VERSION"),
    azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
)
response = client.chat.completions.create(
    model=model,
    messages=messages,
)
response.choices[0].message.content

'Hello! How can I assist you today?'

In [6]:
import os
import openai

messages = [
    {
        "role": "user",
        # "content": "MY NAME IS MINSUK. remember this. I will ask again.",
        "content": "What is my name?",
    },
    # {
    #     "role": "user",
    #     # "content": "MY NAME IS MINSUK. remember this. I will ask again.",
    #     "content": "What is my name?",
    # },
]
model = "gpt-35-turbo"
client = openai.AzureOpenAI(
    api_key=os.environ.get("AZURE_API_KEY"),
    api_version=os.environ.get("API_VERSION"),
    azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
)
response = client.chat.completions.create(
    model=model,
    messages=messages,
    # response_format={"type": "json_object"},
)
response = response.choices[0].message.content
print(response)

KeyboardInterrupt: 

In [ ]:
def apply_llm(
    input_file: str,
    prompt_file: str = "../llm_prompt.txt",
    # num_chunks: int = -1,
    max_tokens: int = 2000,
    model_token_limit: int = 4096,
):
    model = "gpt-3.5-turbo"
    # Initialize the tokenizer
    tokenizer = tiktoken.encoding_for_model(model)

    input = get_string_from_text_file(input_file)
    # prompt = get_string_from_text_file(prompt_file) + input + "```"
    prompt = get_string_from_text_file(prompt_file)
    # Encode the text_data into token integers
    token_integers = tokenizer.encode(input)

    # Split the token integers into chunks based on max_tokens
    # chunk_size = max_tokens - len(token_integers)
    # if num_chunks != -1:
    #     chunk_size = len(token_integers) // num_chunks
    # else:
    #     chunk_size = model_token_limit - 50
    chunks = [
        token_integers[i : i + max_tokens]
        for i in range(0, len(token_integers), max_tokens)
    ]
    # print("chunk size:", max_tokens)

    # Decode token chunks back to strings
    chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print(f"{len(chunks)} chunks processing...")

    responses = []
    messages = [
        {"role": "system", "content": prompt},
        {
            "role": "user",
            "content": "To provide the context for the above prompt, I will send you text in parts. When I am finished, I will tell you 'ALL PARTS SENT'. Do NOT answer until you have received all the parts. You will be penalized otherwise.",
        },
    ]

    for i, chunk in enumerate(chunks):
        messages.append({"role": "user", "content": chunk})

        # Check if total tokens exceed the model's limit and remove oldest chunks if necessary
        while (
            sum(len(tokenizer.encode(msg["content"])) for msg in messages)
            > model_token_limit
        ):
            messages.pop(1)  # Remove the oldest chunk

        # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
        # client.prompt = "\n".join([message["content"] for message in messages])
        # response = client.get_response()
        # responses.append(response)
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo", messages=messages
        )
        # chatgpt_response = response.choices[0].message["content"].strip()
        chatgpt_response = response.choices[0].message.content
        responses.append(chatgpt_response)
        print(f"{i}th chunk processed {responses}")

    # Add the final "ALL PARTS SENT" message
    messages.append({"role": "user", "content": "```\nALL PARTS SENT"})
    # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
    # client.prompt = "\n".join([message["content"] for message in messages])
    # response = client.get_response()
    # responses.append(response)
    response = openai.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
    # final_response = response.choices[0].message["content"].strip()
    final_response = response.choices[0].message.content
    responses.append(final_response)
    return responses

In [ ]:
def apply_llm(
    max_tokens_per_chunk: int = 3500,
    model_token_limit: int = 8192,
    verbose: bool = True,
):
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    input = get_string_from_text_file(
        "../text/3D Convolutional Neural Networks for Human Action Recognition.txt"
    )
    system_prompt = get_string_from_text_file("../llm_prompt.txt")
    # text = input + prompt
    text = input
    token_integers = tokenizer.encode(text)

    print(len(token_integers), "tokens")

    num_chunks = -(-len(token_integers) // max_tokens_per_chunk)
    chunks = []
    print(f"{num_chunks} chunks processing...")
    for i in range(num_chunks):
        start = i * max_tokens_per_chunk
        end = min((i + 1) * max_tokens_per_chunk, len(text))

        chunk = ""
        decoded_chunk = tokenizer.decode(token_integers[start:end])
        if i == num_chunks - 1:
            chunk += (
                f"[START PART {i + 1}/{num_chunks}]\n"
                + text[start:end]
                + f"\n[END PART {i + 1}/{num_chunks}]"
            )
            chunk += "\nALL PARTS SENT. Now you can continue processing the request according to the Instruction."
        else:
            chunk += (
                f'Do not answer yet. This is just another part of the text I want to send you. Just receive and acknowledge as "Part {i + 1}/{num_chunks} received" and wait for the next part.\n[START PART {i + 1}/{num_chunks}]\n'
                + decoded_chunk
                + f"\n[END PART {i + 1}/{num_chunks}]"
            )
            chunk += f'\nRemember not answering yet. Just acknowledge you received this part with the message "Part {i + 1}/{num_chunks} received" and wait for the next part.'

        chunks.append(chunk)

    responses = []

    # model = "gpt-35-turbo"
    # model = "gpt-35-turbo-16k"
    model = "gpt-4"
    client = openai.AzureOpenAI(
        api_key=os.environ.get("AZURE_API_KEY"),
        api_version=os.environ.get("API_VERSION"),
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
    )
    for i, chunk in enumerate(chunks):
        # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
        # client.prompt = chunk
        # response = client.get_response(max_tokens=1000)
        # responses.append(response)
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": chunk,
            },
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format={"type": "json_object"},
        )
        response = response.choices[0].message.content
        responses.append(response)
        if verbose:
            print(response)

    # print(responses)


res = apply_llm()

reading data from ../text/3D Convolutional Neural Networks for Human Action Recognition.txt...
string successfully retrieved.

reading data from ../llm_prompt.txt...
string successfully retrieved.

9047 tokens
2 chunks processing...

 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 

In [ ]:
def apply_llm(
    max_tokens_per_chunk: int = 3400,
    model_token_limit: int = 8192,
    verbose: bool = True,
):
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    input = get_string_from_text_file(
        "../text/3D Convolutional Neural Networks for Human Action Recognition.txt"
    )
    system_prompt = get_string_from_text_file("../llm_prompt.txt")
    # text = input + prompt
    text = input
    token_integers = tokenizer.encode(text)

    print(len(token_integers), "tokens")

    num_chunks = -(-len(token_integers) // max_tokens_per_chunk)
    chunks = []
    print(f"{num_chunks} chunks processing...")
    for i in range(num_chunks):
        start = i * max_tokens_per_chunk
        end = min((i + 1) * max_tokens_per_chunk, len(token_integers))

        chunk = ""
        decoded_chunk = tokenizer.decode(token_integers[start:end])
        chunk += (
            f"[START PART {i + 1}/{num_chunks}. STARTING TOKEN INDEX: {start}]\n"
            + decoded_chunk
            + f"\n[END PART {i + 1}/{num_chunks}. ENDING TOKEN INDEX: {end}]"
        )

        chunks.append(chunk)

    responses = []

    model = "gpt-35-turbo"
    # model = "gpt-35-turbo-16k"
    # model = "gpt-4"
    client = openai.AzureOpenAI(
        api_key=os.environ.get("AZURE_API_KEY"),
        api_version=os.environ.get("API_VERSION"),
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
    )
    for i, chunk in enumerate(chunks):
        # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
        # client.prompt = chunk
        # response = client.get_response(max_tokens=1000)
        # responses.append(response)
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": "\n" + chunk + "\n```\n",
            },
            {
                "role": "user",
                "content": (
                    "Output so far:\n```\n" + responses[-1] + "\n```\n"
                    if i != 0
                    else "no output so far"
                ),
            },
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            # response_format={"type": "json_object"},
        )
        response = response.choices[0].message.content
        responses.append(response)
        if verbose:
            # print(chunk)
            print(response)

    return responses[-1]
    # print(responses)


res = apply_llm()

reading data from ../text/3D Convolutional Neural Networks for Human Action Recognition.txt...
string successfully retrieved.

reading data from ../llm_prompt.txt...
string successfully retrieved.

9047 tokens
3 chunks processing...
{
  "filename": "3D Convolutional Neural Networks for Human Action Recognition",
  "extracted-section-list": [],
  "target-section-extraction-result": {
      "introduction":{
        "start-index": null,
        "end-index": null
      },
      "method":{
        "start-index": null,
        "end-index": null
      },
      "result":{
        "start-index": null,
        "end-index": null
      },
      "conclusion":{
        "start-index": null,
        "end-index": null
      }
  }
}
{
  "filename": "3D Convolutional Neural Networks for Human Action Recognition",
  "extracted-section-list": ["Introduction", "Method", "Result"],
  "target-section-extraction-result": {
      "introduction":{
        "start-index": 0,
        "end-index": 136
      },
     

In [ ]:
# chunk classification
from langchain.prompts import PromptTemplate


def apply_llm(
    max_tokens_per_chunk: int = 400,
    # model_token_limit: int = 8192,
    model_token_limit: int = 4096,
    verbose: bool = True,
):
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    paper_data = get_string_from_text_file(
        "../text/3D Convolutional Neural Networks for Human Action Recognition.txt"
    )
    prompt = get_string_from_text_file("../llm_prompt.txt")
    token_integers = tokenizer.encode(paper_data)

    print(len(token_integers), "tokens")

    num_chunks = -(-len(token_integers) // max_tokens_per_chunk)
    chunks = []
    print(f"{num_chunks} chunks processing...")
    for i in range(num_chunks):
        start = i * max_tokens_per_chunk
        end = min((i + 1) * max_tokens_per_chunk, len(token_integers))

        chunk = ""
        decoded_chunk = tokenizer.decode(token_integers[start:end])
        # chunk += (
        #     f"[START PART {i + 1}/{num_chunks}. STARTING TOKEN INDEX: {start}]\n"
        #     + decoded_chunk
        #     + f"\n[END PART {i + 1}/{num_chunks}. ENDING TOKEN INDEX: {end}]"
        # )

        chunks.append(decoded_chunk)

    responses = []
    model = "gpt-35-turbo"
    # model = "gpt-35-turbo-16k"
    # model = "gpt-4"
    client = openai.AzureOpenAI(
        api_key=os.environ.get("AZURE_API_KEY"),
        api_version=os.environ.get("API_VERSION"),
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
    )
    prev_res = "None"
    possible_sections = ["Introduction", "Method", "Result", "Conclusion", "None"]
    output = {}
    for i, chunk in enumerate(chunks):
        prompt_template = PromptTemplate.from_template(prompt)
        msg = prompt_template.format(
            possible_sections=possible_sections,
            input_chunk=chunk,
            previous_result=prev_res,
            target_chunk_num=i,
            total_chunk_num=num_chunks,
        )
        messages = [
            {
                "role": "user",
                "content": msg,
            },
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            # response_format={"type": "json_object"},
        )
        response = response.choices[0].message.content
        responses.append(response)
        if response != "None":
            output[response] = (output[response] if response in output else "") + chunk

        if response != prev_res:
            print("prev:", prev_res)
            print("new:", response)
            print("-------")
            if prev_res in possible_sections and prev_res != "None":
                possible_sections.remove(prev_res)
                if len(possible_sections) == 1:
                    print(f"All possible sections found. Exiting loop... chunk no.{i}")
                    break

        prev_res = response
        if verbose:
            # print(chunk)
            print(response)

    return output
    # print(responses)


res = apply_llm(verbose=False)

reading data from ../text/3D Convolutional Neural Networks for Human Action Recognition.txt...
string successfully retrieved.

reading data from ../llm_prompt.txt...
string successfully retrieved.

9047 tokens
23 chunks processing...
prev: None
new: Introduction
-------
prev: Introduction
new: Method
-------
prev: Method
new: Result
-------
prev: Result
new: None
-------
prev: None
new: Related Work
-------
prev: Related Work
new: Conclusion
-------
prev: Conclusion
new: None
-------
All possible sections found. Exiting loop... chunk no.12


In [ ]:
hist = []
for paper in os.listdir("../text"):
    text = get_string_from_text_file(paper)
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    length = len(tokenizer.encode(str))
    hist.append(length)
print(sum(hist) / len(hist))

reading data from A frameshift mutation in NOD2 associated with susceptibility to Crohn's disease.txt...
string successfully retrieved.

reading data from Updated world map of the Köppen-Geiger climate classification.txt...
string successfully retrieved.

reading data from A safe operating space for humanity.txt...
string successfully retrieved.

reading data from Mutation in the α-Synuclein Gene Identified in Families with Parkinson's Disease.txt...
string successfully retrieved.

reading data from Capacity of Multi‐antenna Gaussian Channels.txt...
string successfully retrieved.

reading data from Instrumental Variables Regression with Weak Instruments.txt...
string successfully retrieved.

reading data from A Self-Rating Depression Scale.txt...
string successfully retrieved.

reading data from The representative concentration pathways: an overview.txt...
string successfully retrieved.

reading data from Random effects structure for confirmatory hypothesis testing: Keep it maximal.txt

# CODE


In [31]:
import sys
import os

sys.path.append("../vector_db")

In [32]:
from qdrant.lib.llm import LLM, test
import tiktoken
import re
import json
import openai
import time

sys.path.append("../code")
from util import extract_pdf_to_txt, extract_doi_to_txt, get_string_from_text_file

In [101]:
def count_tokens(target: str, model: str = "gpt-4"):
    tokenizer = tiktoken.encoding_for_model(model)
    return len(tokenizer.encode(target))


def handle_exception(e: Exception, log_file_path: str):
    print("Exception occurred during execution. aborting")
    print(e)
    with open(log_file_path, "a") as f:
        f.write("\nAborted due to exception during execution. Error:\n")
        f.write(str(e))
    return False, str(e), "***ERROR DURING EXECUTION***"

In [102]:
def apply_llm(
    paper_txt_path: str = "../text/3D Convolutional Neural Networks for Human Action Recognition.txt",
    model: str = "gpt-4",
) -> tuple[bool, str, str]:
    # load text
    system_prompt = get_string_from_text_file("../prompts/llm_prompt.txt")
    paper_data = get_string_from_text_file(paper_txt_path)
    # make prompt
    prompt = system_prompt + paper_data + "\n```\n"

    # get response from gpt
    llm = LLM(
        base="azure",
        use_model="gpt4" if model == "gpt-4" else "gpt3",
    )
    msgs = [
        {"role": "user", "content": prompt},
        # {"role": "user", "content": "hi give me a response in json"},
    ]
    responses = []

    LOG_DIR = "/Users/ms/cs/ML/MLab/paper-extraction/log"
    RESULT_DIR = "/Users/ms/cs/ML/MLab/paper-extraction/results"
    idx = len(os.listdir(LOG_DIR))
    result_file_name = paper_txt_path.split("/")[-1].replace(".txt", "")
    output_log_file_path = os.path.join(LOG_DIR, f"LOG-{idx}_{result_file_name}.log")
    output_result_file_path = os.path.join(RESULT_DIR, result_file_name + ".json")
    total_input_token, total_output_token = 0, 0
    print("----------------------------------------------")
    print(f"Paper title: {result_file_name}")
    print(f"Paper contains {count_tokens(paper_data,model=model)} tokens")
    start_time = time.time()
    try:
        while True:
            llm.set_prompt(prompt=msgs)
            response, finish_reason, usage = llm.get_response(
                max_tokens=4096,
                # max_tokens=100,
                response_format="text",
                # response_format="json_object",
                is_custom=True,
            )
            # write log
            with open(output_log_file_path, "a") as f:
                f.write(response)
                f.write(f"\n\nfinish reason: {finish_reason}")
                f.write(f"\ntoken usage{str(usage)}")
                f.write("\n=======================================\n")

            print("Tokens used:", usage, "Finish reason:", finish_reason)
            responses.append(response.replace("\n", " "))
            total_input_token += usage.prompt_tokens
            total_output_token += usage.completion_tokens
            if finish_reason == "stop":
                break
            elif finish_reason == "length":
                msgs.append({"role": "assistant", "content": response})
                msgs.append(
                    {
                        "role": "user",
                        "content": "Your response was cut off. Please CONTINUE FROM WHERE YOU STOPPED. Your response will be appended to your previous response, and thus it should be a continuation of your previous response. Because it is a continuation, your response may not be in JSON format. However, note that the final result (all your responses added up) need to be in JSON format (This is important). Refer to the output format to see where you left off. You will be penalized for starting your response from the beginning, or outputting inconsistent responses",
                    }
                )
                continue
            else:
                raise Exception(
                    "Finish reason of gpt response needs to be checked. Finish reason:",
                    finish_reason,
                )
    except Exception as e:
        return handle_exception(e, output_log_file_path)

    print(f"{len(responses)} API calls")
    final_response = "".join(responses)
    num_tokens = count_tokens(final_response, model=model)
    print(f"final response is {num_tokens} tokens")
    print(f"Total input tokens used:{total_input_token}")
    print(f"Total output tokens used:{total_output_token}")
    # check if valid json
    try:
        final_response = re.findall(r"\{.*\}", final_response, re.DOTALL)[0]
        json.loads(final_response)
    except Exception as e:
        is_valid_json = False
        return handle_exception(e, output_log_file_path)
    else:
        is_valid_json = True
    # is_valid_json = type(json.loads(final_response)) is dict
    print(f"Is valid json: {is_valid_json}")
    with open("../gpt_response.txt", "w") as file:
        # file.write(response)
        file.write(final_response)

    # return response
    with open(output_log_file_path, "a") as f:
        f.write(f"\n\nFINAL RESPONSE ({num_tokens} tokens):\n")
        f.write(final_response)
        f.write(f"\n\nTotal input tokens:{total_input_token}")
        f.write(f"\nTotal output tokens:{total_output_token}")
        f.write(f"\nIs valid JSON:{is_valid_json}")
        f.write(f"\nTime elapsed: {time.time()-start_time}")
    with open(output_result_file_path, "w") as f:
        f.write(final_response)
    return True, "-", final_response


# is_pass, e, res = apply_llm()
# print(res)

In [99]:
with open(
    "/Users/ms/cs/ML/MLab/paper-extraction/results/3D Convolutional Neural Networks for Human Action Recognition.json",
    "r",
) as f:
    print(json.dumps(json.load(f), indent=4))

{
    "filename": "3D Convolutional Neural Networks for Human Action Recognition",
    "extracted-section-list": [
        "Introduction",
        "3D Convolutional Neural Networks",
        "Related Work",
        "Experiments",
        "Conclusions and Discussions"
    ],
    "target-section-extraction-result": {
        "introduction": "Recognizing human actions in real-world environment\n\u00ads applications in a variety of domains including in-\ntelligent video surveillance, customer attributes, and\nshopping behavior analysis. However, accurate recog-\nnition of actions is a highly challenging task due to\nAppearing in Proceedings of the 27thInternational Confer-\nence on Machine Learning , Haifa, Israel, 2010. Copyright\n2010 by the author(s)/owner(s).cluttered backgrounds, occlusions, and viewpoint vari-\nations, etc. Therefore, most of the existing approaches\n(Efros et al. ,2003;Sch\u0308 uldt et al. ,2004;Doll\u0301 ar et al. ,\n2005;Laptev & P\u0301 erez ,2007;Jhuang et al.

In [104]:
from util import list_to_text_file, get_string_from_text_file

TEXT_DIR = "/Users/ms/cs/ML/MLab/paper-extraction/text"
NOTEBOOK_DIR = "/Users/ms/cs/ML/MLab/paper-extraction/notebook"
evaluation_list_filepath = os.path.join(NOTEBOOK_DIR, "llm_extraction_paper_list.txt")
exception_list_filepath = os.path.join(
    NOTEBOOK_DIR, "llm_extraction_paper_exception_list.txt"
)
evaluation_list = get_string_from_text_file(evaluation_list_filepath).splitlines()
exception_list = get_string_from_text_file(exception_list_filepath).splitlines()
evaluation_list = [paper for paper in evaluation_list if paper not in exception_list]


pass_cnt = 0
fail_cnt = 0
skip_cnt = 0
MAX_NUM_OF_PAPERS_FOR_EVAL = 50
# erase extraction paper list if you want to restart
for paper in os.listdir(TEXT_DIR):
    if (pass_cnt + fail_cnt + skip_cnt) >= MAX_NUM_OF_PAPERS_FOR_EVAL:
        print("===== result =====")
        print(f"Passed: {pass_cnt}")
        print(f"Failed: {fail_cnt}")
        print(f"Skipped: {skip_cnt}")
        break

    if paper in evaluation_list or paper in exception_list:
        print(f"Skipping as paper in evaluation/exception list. \nPaper: {paper}\n")
        skip_cnt += 1
        continue

    is_pass, e, res = apply_llm(paper_txt_path=paper)
    print("Extraction Result:", "Pass" if is_pass else "Fail")
    if is_pass:
        evaluation_list.append(paper)
        pass_cnt += 1
    else:
        exception_list.append(paper)
        fail_cnt += 1

list_to_text_file(evaluation_list, evaluation_list_filepath)
list_to_text_file(exception_list, exception_list_filepath)

----------------------------------------------
Paper title: A frameshift mutation in NOD2 associated with susceptibility to Crohn's disease
Paper contains 8874 tokens
Exception occurred during execution. aborting
Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 24 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}
Extraction Result: Fail
----------------------------------------------
Paper title: Updated world map of the Köppen-Geiger climate classification
Paper contains 15311 tokens
Exception occurred during execution. aborting
Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded token rate limit of your current OpenAI S0 

# Show results on google drive


In [83]:
import gspread
from typing import Literal

# connect to the service account
gc = gspread.service_account(filename="../creds.json")
sh = gc.open("Paper Extraction v2")
# sh1 = sh.get_worksheet(0)
sh2 = sh.get_worksheet(1)

In [94]:
def gs_update_batch(
    batch_list_file_name: str = "../evaluation_paper_list",
    extraction_method: Literal["regex", "llm"] = "llm",
    start_row_num: int = 4,
    sheet=sh2,
):
    JSON_DIR = "/Users/ms/cs/ML/MLab/paper-extraction/results"
    # get paper list
    # papers = get_string_from_text_file(batch_list_file_name).splitlines()
    # exception_list = get_string_from_text_file(
    # "../evaluation_paper_exception_list.txt"
    # ).splitlines()
    values = []
    results = os.listdir(JSON_DIR)
    # print(len(results))
    for filename in results:
        # if filename in exception_list:
        #     continue
        # content = get_string_from_text_file(filename)
        # res = extract(content, filename, extraction_method)

        with open(os.path.join(JSON_DIR, filename), "r") as f:
            res = json.load(f)
        temp = res["target-section-extraction-result"]
        values.append(
            [
                res["filename"],
                "\n".join(res["extracted-section-list"]),
                temp["introduction"],
                len(temp["introduction"]),
                len(temp["introduction"].split()),
                "null",
                temp["method"],
                len(temp["method"]),
                len(temp["method"].split()),
                "null",
                temp["result"],
                len(temp["result"]),
                len(temp["result"].split()),
                "null",
                temp["conclusion"],
                len(temp["conclusion"]),
                len(temp["conclusion"].split()),
                "null",
            ]
        )

    sheet.batch_update(
        [
            {
                "range": f"A{start_row_num}:R{start_row_num-1+len(results)}",
                "values": values,
            }
        ]
    )

In [95]:
gs_update_batch()

# Token number for papers


In [50]:
file_path = "/Users/ms/cs/ML/MLab/paper-extraction/text/Advances in functional and structural MR image analysis and implementation as FSL.txt"
token_num = len(tokenizer.encode(get_string_from_text_file(file_path)))
token_num

15551

In [54]:
TEXT_DIR = "/Users/ms/cs/ML/MLab/paper-extraction/text"
cnt = 0
max = 20
tokenizer = tiktoken.encoding_for_model("gpt-4")
for paper in os.listdir(TEXT_DIR):
    print(paper)
    token_num = len(tokenizer.encode(get_string_from_text_file(paper)))
    print(token_num)
    # apply_llm(paper_txt_path=paper)
    cnt += 1
    if cnt >= max:
        break

A frameshift mutation in NOD2 associated with susceptibility to Crohn's disease.txt
8874
Updated world map of the Köppen-Geiger climate classification.txt
15311
A safe operating space for humanity.txt
7425
Mutation in the α-Synuclein Gene Identified in Families with Parkinson's Disease.txt
6715
Capacity of Multi‐antenna Gaussian Channels.txt
70492
Instrumental Variables Regression with Weak Instruments.txt
0
A Self-Rating Depression Scale.txt
36839
The representative concentration pathways: an overview.txt
22977
Random effects structure for confirmatory hypothesis testing: Keep it maximal.txt
30237
Homeostasis model assessment: insulin resistance and ?-cell function from fasting plasma glucose and insulin concentrations in man.txt
9638
Gene expression profiling predicts clinical outcome of breast cancer.txt
76080
A comparison of methods to test mediation and other intervening variable effects..txt
23713
Opinion Mining and Sentiment Analysis.txt
70416
Rich Feature Hierarchies for Accura